In [1]:
import torch
import numpy as np


import os
import glob
import re
import pandas


/tmp/ipykernel_34978/2016042400.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [2]:
prefix = '/mnt/hwfile/share_data/suzhongling/Ai4S/results/16T/open_CALcomplexHalf_COMint4kernel_TUNE1/Nodes32/ntask0/'
data2 = torch.cat([torch.load(prefix + f'/rank{rank}.pt') for rank in range(256)])

In [3]:
data2.shape

torch.Size([2999996, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [4]:
bitstringopen = torch.load('./sc41_reproduce_scheme_n53_m20_ABCDCDAB_3000000_einsum_13_open.pt')[-1]

In [5]:
len(bitstringopen)

2999996

In [6]:
permute_idx = [7, 3, 12, 0, 4, 5, 6, 10, 8, 11, 9, 1, 2] # this is the permutation index of open result
open_qubits = sorted([11, 12, 13, 19, 20, 22, 23, 28, 33, 37, 38, 40, 44])

In [7]:
close_qubits = list(set(range(53)) - set(open_qubits))

In [8]:
def find_corresponding_indices(dig, dig_raw_pre):
    mapping = {}
    for index, num in enumerate(dig):
        mapping[num] = index

    result = []
    for num in dig_raw_pre:
        assert num in mapping
        result.append(mapping[num])

    return result

In [9]:
import pandas as pd
import numpy as np

# 读取文件，设定分隔符为 "\t"
df = pd.read_csv("amps3M_all.txt", sep="\t", header=None)

def parse_complex(string):
    string = string.strip('()')  # remove parentheses
    real, imag = map(float, string.split(','))  # split on comma and convert to float
    return complex(real, imag)  # return complex number

# 将第二列的字符串转换成复数
df[1] = df[1].apply(lambda x: parse_complex(x))

# 为列命名
df.columns = ["Binary", "Complex", "Float"]
print(df)

                                                    Binary  \
0        0000000000011000000000110000100101100100010010...   
1        0000000000001100000011001000110000100101100000...   
2        0000000000010000000011101100000111100100100010...   
3        0000000000000100000011001100100000000110110100...   
4        0000000000011000000101011100110001100010110111...   
...                                                    ...   
2999995  1111111111110011001010000011111001011001001100...   
2999996  1111111111101011011110100011011001111101101011...   
2999997  1111111111111111011011000111000001111111000000...   
2999998  1111111111111111011101011111111111111011101100...   
2999999  1111111111111111101000010111101010111100010110...   

                                                Complex         Float  
0        9.342210e-09-4.777170e-                    10j  8.750510e-17  
1       -3.582660e-10+3.278370e-                    09j  1.087606e-17  
2        1.348300e-08+7.413220e-       

In [10]:
df["BinaryClose"] = df["Binary"].apply(lambda x: "".join(np.array(list(x))[close_qubits]))

In [11]:
df["Closeidx"] = df["Binary"].apply(lambda x: 
                                   np.frombuffer(bytes(x,'ascii'), dtype=np.uint8)[open_qubits][permute_idx]
                                             - ord('0'))

In [16]:
corresponding_indices = find_corresponding_indices(bitstringopen,df["BinaryClose"])

In [17]:
len(corresponding_indices)

3000000

In [18]:
amp_exact = df["Complex"].values

In [19]:
amp_exact

array([ 9.34221e-09-4.77717e-10j, -3.58266e-10+3.27837e-09j,
        1.34830e-08+7.41322e-09j, ..., -5.19345e-09-3.29486e-10j,
       -1.46572e-08+1.16419e-08j,  4.50606e-09+4.84449e-09j])

In [21]:
index_array = np.array(corresponding_indices).reshape(-1)
closeidx = (np.stack(df["Closeidx"].values)*np.array([2**(len(open_qubits)-x-1) for x in range(len(open_qubits))])).sum(axis=-1)
index_array = index_array*(2**len(open_qubits)) + closeidx

In [22]:
index_array

array([       5846,       12559,       18332, ..., 21737898807,
       21737912955, 23185765959])

In [23]:
data2.flatten().shape

torch.Size([24575967232])

In [24]:
amp_sel = data2.flatten().index_select(0, torch.from_numpy(index_array))

In [25]:
amplitude_exact = torch.tensor(amp_exact.copy(),dtype=torch.complex64)[:]
amplitude_appro = amp_sel.clone().to(torch.complex64)[:]
fidelity = (
    (amplitude_exact.conj() @ amplitude_appro.reshape(-1)).abs() /
    (amplitude_exact.abs().square().sum().sqrt() * amplitude_appro.abs().square().sum().sqrt())
).square().item()
fidelity

0.0002394422044744715

In [101]:
10/2**18

3.814697265625e-05

In [57]:
amp_exact[::-1]

array([ 4.50606e-09+4.84449e-09j, -1.46572e-08+1.16419e-08j,
       -5.19345e-09-3.29486e-10j, ...,  1.34830e-08+7.41322e-09j,
       -3.58266e-10+3.27837e-09j,  9.34221e-09-4.77717e-10j])

In [110]:
0.00027065412723459303*np.log(1024)/76*82

0.002024139198205896

In [26]:
0.0002394422044744715*np.log(2**13)

0.002157592956200992